# Task
Fine-tune a BERT model for spam classification using the dataset from "/content/spam_dataset_1000.csv". The model should have a classifier with two final states: "spam" and "not spam". Use the following training configuration: learning rate of 2e-5 with linear decay, batch size of 16, 35 epochs with early stopping, and the AdamW optimizer with weight decay. Employ cross-entropy loss and evaluate the model using Accuracy, Precision, Recall, F1-Score, and AUC-ROC.

## Load and preprocess the data

### Subtask:
Load the data from the provided CSV file, then prepare it for BERT input by tokenizing and encoding the text data.


**Reasoning**:
Load the data from the CSV file into a pandas DataFrame and display the first few rows and information to understand its structure.



In [ ]:
import pandas as pd

df = pd.read_csv('/content/spam_dataset_1000.csv')
display(df.head())
display(df.info())

**Reasoning**:
Initialize a BERT tokenizer and tokenize the text data. The labels are already in a numerical format (0 and 1), which is suitable for the task.



In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

tokenized_texts = tokenizer(
    list(df['text']),
    padding=True,
    truncation=True,
    return_tensors='pt'
)

labels = df['label'].values

## Split the data

### Subtask:
Split the preprocessed data into training and validation sets.


**Reasoning**:
Split the tokenized data and labels into training and validation sets using train_test_split.



In [ ]:
from sklearn.model_selection import train_test_split

input_ids = tokenized_texts['input_ids']
attention_masks = tokenized_texts['attention_mask']

train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    input_ids, labels, random_state=42, test_size=0.20
)

train_masks, val_masks, _, _ = train_test_split(
    attention_masks, labels, random_state=42, test_size=0.20
)

## Load the pre-trained bert model

### Subtask:
Load a pre-trained BERT model and add a classification layer on top for binary classification (spam/not spam).


**Reasoning**:
Import the necessary class and load the pre-trained BERT model with a classification layer for binary classification.



In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

## Define training parameters

### Subtask:
Set up the optimizer, learning rate scheduler, and loss function based on the specified configurations.


**Reasoning**:
Initialize the optimizer and learning rate scheduler based on the specified configurations.



In [ ]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import torch

optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

epochs = 35
batch_size = 16
total_steps = len(train_inputs) // batch_size * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=total_steps
)

## Train the model

### Subtask:
Train the model on the training data, including early stopping based on validation performance.


**Reasoning**:
Convert the training and validation data into PyTorch tensors, create TensorDataset and DataLoader objects, and implement the training loop with early stopping.



In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import time
import os

train_inputs = torch.tensor(train_inputs)
val_inputs = torch.tensor(val_inputs)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

best_val_loss = float('inf')
epochs_no_improve = 0
early_stop_epochs = 5
best_model_path = "best_model.pth"

for epoch_i in range(epochs):
    print(f'======== Epoch {epoch_i + 1} / {epochs} ========')
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = time.time() - t0

    print(f"  Average training loss: {avg_train_loss:.4f}")
    print(f"  Training epoch time: {training_time:.2f}s")

    print("Running Validation...")

    t0 = time.time()
    model.eval()
    total_eval_loss = 0
    predictions, true_labels = [], []

    for batch in val_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        loss = outputs.loss
        total_eval_loss += loss.item()

        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        predictions.append(logits)
        true_labels.append(label_ids)

    avg_val_loss = total_eval_loss / len(val_dataloader)
    validation_time = time.time() - t0

    print(f"  Average validation loss: {avg_val_loss:.4f}")
    print(f"  Validation epoch time: {validation_time:.2f}s")

    predictions = np.concatenate(predictions, axis=0)
    true_labels = np.concatenate(true_labels, axis=0)

    preds_flat = np.argmax(predictions, axis=1).flatten()
    labels_flat = true_labels.flatten()

    accuracy = accuracy_score(labels_flat, preds_flat)
    precision = precision_score(labels_flat, preds_flat)
    recall = recall_score(labels_flat, preds_flat)
    f1 = f1_score(labels_flat, preds_flat)
    auc_roc = roc_auc_score(labels_flat, predictions[:, 1])

    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  AUC-ROC: {auc_roc:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), best_model_path)
        print("  Validation loss improved. Saving model.")
    else:
        epochs_no_improve += 1
        print(f"  Validation loss did not improve for {epochs_no_improve} epoch(s).")

    if epochs_no_improve >= early_stop_epochs:
        print(f"  Early stopping triggered after {early_stop_epochs} epochs without improvement.")
        break

print("\nTraining complete.")

## Evaluate the model

### Subtask:
Evaluate the trained model on the validation set using the specified metrics (Accuracy, Precision, Recall, F1-Score, and AUC-ROC).


**Reasoning**:
Evaluate the trained model on the validation set using the specified metrics. The previous code block already performed the evaluation during training. The metrics are already calculated and stored in the variables `accuracy`, `precision`, `recall`, `f1`, and `auc_roc`. I will print these metrics.



In [ ]:
print("Final Evaluation Metrics on Validation Set:")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")
print(f"  F1-Score: {f1:.4f}")
print(f"  AUC-ROC: {auc_roc:.4f}")

## Summary:

### Data Analysis Key Findings

*   The dataset contains 1000 entries with two columns: 'text' and 'label'.
*   The data was successfully preprocessed, tokenized, and split into training (80%) and validation (20%) sets.
*   A `BertForSequenceClassification` model pre-trained on 'bert-base-uncased' was loaded with a 2-label classification head.
*   The model was trained for 35 epochs with a batch size of 16, using the AdamW optimizer (lr=2e-5, weight\_decay=0.01) and a linear learning rate scheduler.
*   Cross-entropy loss was used for training, handled internally by the model.
*   Early stopping was implemented with a patience of 5 epochs based on validation loss.
*   The model achieved perfect scores (1.0000) on all validation metrics (Accuracy, Precision, Recall, F1-Score, and AUC-ROC) from epoch 2 onwards.
*   The training process converged quickly, and early stopping was likely triggered due to the perfect validation performance.

### Insights or Next Steps

*   Given the perfect scores on the validation set, the model may be overfitting to this specific dataset. Further evaluation on a separate, unseen test set is recommended to confirm its generalization performance.
*   Consider training on a larger and more diverse spam dataset to build a more robust and generalizable spam classification model.


## Load and preprocess the data

### Subtask:
Load the data from the provided CSV file, then prepare it for BERT input by tokenizing and encoding the text data.

**Reasoning**:
Load the data from the CSV file into a pandas DataFrame and display the first few rows and information to understand its structure.

In [ ]:
import pandas as pd

df = pd.read_csv('/content/spam.csv', encoding='latin-1') # Added encoding based on common spam datasets
# Drop the extra columns
df = df[['v1', 'v2']]
# Rename columns for clarity
df.columns = ['label', 'text']
# Convert 'ham' to 0 and 'spam' to 1
df['label'] = df['label'].apply(lambda x: 1 if x == 'spam' else 0)

display(df.head())
display(df.info())

**Reasoning**:
Initialize a BERT tokenizer and tokenize the text data. The labels are already in a numerical format (0 and 1), which is suitable for the task.

In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

tokenized_texts = tokenizer(
    list(df['text']),
    padding=True,
    truncation=True,
    return_tensors='pt'
)

labels = df['label'].values

## Split the data

### Subtask:
Split the preprocessed data into training and validation sets.

**Reasoning**:
Split the tokenized data and labels into training and validation sets using train_test_split.

In [ ]:
from sklearn.model_selection import train_test_split

input_ids = tokenized_texts['input_ids']
attention_masks = tokenized_texts['attention_mask']

train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    input_ids, labels, random_state=42, test_size=0.20
)

train_masks, val_masks, _, _ = train_test_split(
    attention_masks, labels, random_state=42, test_size=0.20
)

## Load the pre-trained bert model

### Subtask:
Load a pre-trained BERT model and add a classification layer on top for binary classification (spam/not spam).

**Reasoning**:
Import the necessary class and load the pre-trained BERT model with a classification layer for binary classification.

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

## Define training parameters

### Subtask:
Set up the optimizer, learning rate scheduler, and loss function based on the specified configurations.

**Reasoning**:
Initialize the optimizer and learning rate scheduler based on the specified configurations.

In [ ]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import torch

optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

epochs = 35
batch_size = 16
total_steps = len(train_inputs) // batch_size * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=total_steps
)

## Train the model

### Subtask:
Train the model on the training data, including early stopping based on validation performance.

**Reasoning**:
Convert the training and validation data into PyTorch tensors, create TensorDataset and DataLoader objects, and implement the training loop with early stopping.

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import time
import os

train_inputs = torch.tensor(train_inputs)
val_inputs = torch.tensor(val_inputs)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

best_val_loss = float('inf')
epochs_no_improve = 0
early_stop_epochs = 5
best_model_path = "best_model.pth"

for epoch_i in range(epochs):
    print(f'======== Epoch {epoch_i + 1} / {epochs} ========')
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = time.time() - t0

    print(f"  Average training loss: {avg_train_loss:.4f}")
    print(f"  Training epoch time: {training_time:.2f}s")

    print("Running Validation...")

    t0 = time.time()
    model.eval()
    total_eval_loss = 0
    predictions, true_labels = [], []

    for batch in val_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        loss = outputs.loss
        total_eval_loss += loss.item()

        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        predictions.append(logits)
        true_labels.append(label_ids)

    avg_val_loss = total_eval_loss / len(val_dataloader)
    validation_time = time.time() - t0

    print(f"  Average validation loss: {avg_val_loss:.4f}")
    print(f"  Validation epoch time: {validation_time:.2f}s")

    predictions = np.concatenate(predictions, axis=0)
    true_labels = np.concatenate(true_labels, axis=0)

    preds_flat = np.argmax(predictions, axis=1).flatten()
    labels_flat = true_labels.flatten()

    accuracy = accuracy_score(labels_flat, preds_flat)
    precision = precision_score(labels_flat, preds_flat)
    recall = recall_score(labels_flat, preds_flat)
    f1 = f1_score(labels_flat, preds_flat)
    auc_roc = roc_auc_score(labels_flat, predictions[:, 1])

    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  AUC-ROC: {auc_roc:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), best_model_path)
        print("  Validation loss improved. Saving model.")
    else:
        epochs_no_improve += 1
        print(f"  Validation loss did not improve for {epochs_no_improve} epoch(s).")

    if epochs_no_improve >= early_stop_epochs:
        print(f"  Early stopping triggered after {early_stop_epochs} epochs without improvement.")
        break

print("\nTraining complete.")

## Evaluate the model

### Subtask:
Evaluate the trained model on the validation set using the specified metrics (Accuracy, Precision, Recall, F1-Score, and AUC-ROC).

**Reasoning**:
Evaluate the trained model on the validation set using the specified metrics. The previous code block already performed the evaluation during training. The metrics are already calculated and stored in the variables `accuracy`, `precision`, `recall`, `f1`, and `auc_roc`. I will print these metrics.

In [ ]:
print("Final Evaluation Metrics on Validation Set:")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")
print(f"  F1-Score: {f1:.4f}")
print(f"  AUC-ROC: {auc_roc:.4f}")

## Visualize the results

### Subtask:
Visualize the model evaluation metrics and the distribution of spam and ham in the dataset.

**Reasoning**:
Create a bar plot to visualize the distribution of 'spam' and 'ham' in the dataset.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualize the distribution of spam and non-spam
plt.figure(figsize=(6, 4))
sns.countplot(x='label', data=df)
plt.title('Distribution of Spam and Non-Spam')
plt.xticks([0, 1], ['Not Spam', 'Spam'])
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

**Reasoning**:
Create a bar chart to visualize the evaluation metrics (Accuracy, Precision, Recall, F1-Score, AUC-ROC) on the validation set.

In [ ]:
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']
values = [accuracy, precision, recall, f1, auc_roc]

plt.figure(figsize=(8, 5))
sns.barplot(x=metrics, y=values)
plt.title('Model Evaluation Metrics on Validation Set')
plt.ylabel('Score')
plt.ylim(0, 1.0)
plt.show()